- TensorFlow 中文主页: https://tensorflow.google.cn/

下面主要介绍 [Low Level APIs](https://tensorflow.google.cn/guide/low_level_intro)：

## TensorFlow Core Walkthrough

一般地，TensorFlow 编程由以下两个独立的部分组成：

- 通过 `tf.Graph` 建立计算图 (Building the computational graph (a `tf.Graph`).)
- 使用 `tf.Session` 运行计算图(Running the computational graph (using a `tf.Session`).)

In [2]:
import numpy as np
import tensorflow as tf

## Graph 与 Session

A computational graph is a series of TensorFlow operations arranged into a graph. The graph is composed of two types of objects.

- `tf.Operation` (or "ops"): The nodes of the graph. Operations describe calculations that consume and produce tensors.
- `tf.Tensor`: The edges in the graph. These represent the values that will flow through the graph. Most TensorFlow functions return tf.Tensors.

计算图是由一系列 TensorFlow 的运算 (ops) 按照一定的顺序排列构成的。在一个 `tf.Graph` 中包含两种类型的对象：

- `tf.Operation` (or "ops"): 图的节点（nodes）. 节点描述了如何 对 Tensor 的输入和输出进行计算；
- `tf.Tensor`: 图的边（edges）. 边代表了流经计算图的值。

大多数 TensorFlow 函数均返回 `tf.Tensors`.

**重要: `tf.Tensors` 并不存储 values, 而它仅仅是处理计算图中元素的一个 handle.**

In [4]:
a = tf.constant(3.0, dtype=tf.float32)
b = tf.constant(4.0)  # also tf.float32 implicitly
total = a + b
print(a)
print(b)
print(total)

Tensor("Const:0", shape=(), dtype=float32)
Tensor("Const_1:0", shape=(), dtype=float32)
Tensor("add:0", shape=(), dtype=float32)


上面说的不是很好理解，我说下自己的理解：

- `tf.Tensor` 可以理解为一个符号，比如数学中的未知元 $x$ (仅仅是一个**代表**, 它可以是数组，字符串等)
- `tf.Operation` 可以理解为 $x$ 的方程，比如 $x^7 + 3x^5 + 1$ (只要你可以定义一个合理的运算规则, 若 $x$ 是字符串，该方程也可能成立)

比如，当你传入 $x=1$，则会输出运算的结果 $x^7 + 3x^5 + 1 = 10$；当没有值传递给 $x$ 时，它们仅仅是一串符号而已。

换句话说，`tf.Graph` 是一个定义了各种 `tf.Tensor` 的依赖关系 (`tf.Operation`) 抽象计算图，而 `tf.Session` 可以看作是 `tf.Graph` 的一个实例（具象化）。

## Datasets

对于一些简单的实验，使用 `tf.placeholder` 即可 feed 数据。但是，对于数据量比较大或者实验比较复杂，则使用迭代器是一个很好的选择：

- To get a runnable `tf.Tensor` from a Dataset you must first convert it to a `tf.data.Iterator`, and then call the Iterator's `tf.data.Iterator.get_next` method.

- The simplest way to create an Iterator is with the `tf.data.Dataset.make_one_shot_iterator` method. For example, in the following code the next_item tensor will return a row from the my_data array on each run call:

In [6]:
my_data = [
    [0, 1, ],
    [2, 3, ],
    [4, 5, ],
    [6, 7, ],
]
slices = tf.data.Dataset.from_tensor_slices(my_data)
next_item = slices.make_one_shot_iterator().get_next()

In [7]:
next_item

<tf.Tensor 'IteratorGetNext:0' shape=(2,) dtype=int32>

注意，当此迭代器 Costume 尽所有数据时，你再运行 `.get_next()` 会报错，为此，TensorFlow 提供了一个异常处理机制：` tf.errors.OutOfRangeError`, 下面是一个使用的例子：

```python
while True:
    try:
        print(sess.run(next_item))
    except tf.errors.OutOfRangeError:
        break
```

如果 Dataset 依赖于节点的状态，那么你需要在使用迭代器之前对其进行初始化：

```py
r = tf.random_normal([10, 3])
dataset = tf.data.Dataset.from_tensor_slices(r)
iterator = dataset.make_initializable_iterator()
next_row = iterator.get_next()

sess.run(iterator.initializer)
while True:
    try:
        print(sess.run(next_row))
    except tf.errors.OutOfRangeError:
        break
```

详细见[datasets](https://tensorflow.google.cn/guide/datasets)

## Layers

使用 [`tf.layers`](https://tensorflow.google.cn/api_docs/python/tf/layers) 将需要训练的参数投入到计算图中是一个很好的选择。

下面我们使用 `tf.layers` 创建一个线性模型：

In [8]:
x = tf.constant([[1], [2], [3], [4]], dtype=tf.float32)
y_true = tf.constant([[0], [-1], [-2], [-3]], dtype=tf.float32)

linear_model = tf.layers.Dense(units=1)

y_pred = linear_model(x)
loss = tf.losses.mean_squared_error(labels=y_true, predictions=y_pred)

optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

init = tf.global_variables_initializer()

sess = tf.Session()
sess.run(init)
for i in range(100):
    _, loss_value = sess.run((train, loss))
    print(loss_value)

print(sess.run(y_pred))

2.915727
2.0932546
1.5221393
1.1254379
0.84976107
0.6580632
0.524639
0.43165198
0.36672604
0.32127345
0.28933543
0.26677722
0.2507299
0.2392027
0.2308142
0.224606
0.21991292
0.21627343
0.21336734
0.21097241
0.20893447
0.20714638
0.20553397
0.20404565
0.20264567
0.20130913
0.20001884
0.19876285
0.19753277
0.19632277
0.19512892
0.19394833
0.19277908
0.19161972
0.19046944
0.1893274
0.18819323
0.18706644
0.18594694
0.18483445
0.18372881
0.18263
0.18153784
0.18045224
0.17937325
0.17830072
0.17723463
0.17617494
0.17512155
0.17407453
0.17303377
0.17199922
0.17097089
0.16994862
0.16893259
0.1679225
0.16691852
0.16592054
0.16492856
0.16394243
0.16296226
0.16198795
0.16101941
0.16005675
0.15909976
0.15814856
0.15720297
0.15626311
0.15532887
0.15440014
0.15347703
0.1525594
0.15164725
0.1507406
0.14983936
0.1489435
0.14805298
0.14716777
0.1462879
0.14541325
0.14454384
0.14367963
0.14282064
0.14196673
0.1411179
0.1402742
0.1394355
0.13860185
0.13777319
0.13694946
0.13613066
0.13531674
0.1345077
0.1